# Шаг 0.
Собираем сам датасет для парсинга:
1. Переходим в чатик (используем клиент ТГ для ПК)
2. Кликаем на "три точки" -> Экспорт истории чата
3. Заменяем название папки в коде на свою
4. Изменяем количество HTML-файлов(в моём случае 22) на своё количество

 # Собираем датасет в один список

In [43]:
import pandas as pd
msgs = []
html_files_cnt = 22
folder = "ChatExport_2024-10-31"
ds = pd.DataFrame()

In [25]:
# Оч классная библиотека, указываешь текст, который содержится в одном из необходимых элементов, и она тебе парсит все подобные элементы
from autoscraper import AutoScraper
wanted_list = ["Доброе утро, парни и непарни"]
scraper = AutoScraper()
htmls = open(f"{folder}/messages.html").read()
#print(htmls)
result = scraper.build(wanted_list=wanted_list, html=htmls)
msgs+=result
#scraper.save("tg-msgs-parser")

In [26]:
msgs

['привет',
 'Привет',
 'Привет всем)',
 'Test',
 'Доброе утро, парни и непарни',
 'Hello',
 'Дороу',
 'Шо Стамбул пасаны?',
 'Всем приветНу что народ, скидываемся на ИП в Грузии',
 'прилечу через месяц',
 'А шо там по ип?',
 'А оформляешь ИП - тебе дают разрешение на проживание',
 'если есть гайд - скинь',
 'Живешь там сколько-то лет (вроде лет 6), и вуаля - теперь ты гражданин Грузии',
 'В цифрах мог ошибиться, может, и 8 лет',
 'Страно шо Мексики нет',
 'Хаха, нет, ни методички, ни бабки-лабки к этой лабораторной нет',
 'А кто-то есть, кто проходит аспу в России, но тоже думает релоцироваться?',
 'действую по приборам',
 'Я',
 'мне письма пишут: "почему не принес отчет по нир"',
 'Есть такие мысли',
 'Ну всем привет',
 'Ай, блин, ссылка без истории',
 'Короче, вопрос, наверное, больше про армейкуТипа, если уезжать, то это до окончания призывного возраста без возможности заехать и сказать маме с папой "Hello"',
 'Да',
 'Настройте плиз, чтобы историю было видно',
 '+',
 'Если к

## Теперь можем использовать нашу модельку для парсинга остальных файлов

In [17]:
scraper.load("tg-msgs-parser")

In [27]:
for i in range(2, html_files_cnt):
    msgs+=scraper.get_result(html=open(f"{folder}/messages{i}.html").read())

In [37]:
# У библиотеки похоже какая-то бага, иногда списки получаются вложенными, нужно их распаковать
def flatten_list(lst):
    flattened = []
    for item in lst:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

In [38]:
new_msgs = flatten_list(msgs)

In [39]:
print(len(new_msgs))

14706


In [44]:
ds['messages'] = new_msgs

In [46]:
ds.to_csv('phystech_relocation_messages.csv')

In [47]:
!ls

 analytics.ipynb
 ChatExport_2024-10-31
 phystech_relocation_messages.csv
 tg-msgs-parser
'Передать посылку или привезти из-за границы (24.10.2024 17_25_16).html'
'Передать посылку или привезти из-за границы3.html'


# Делаем эмбеддинги

In [ ]:
import requests
import numpy as np
from scipy.spatial.distance import cdist

FOLDER_ID = "<идентификатор_каталога>"
IAM_TOKEN = "<IAM-токен>"

doc_uri = f"emb://{FOLDER_ID}/text-search-doc/latest"
query_uri = f"emb://{FOLDER_ID}/text-search-query/latest"

embed_url = "https://llm.api.cloud.yandex.net:443/foundationModels/v1/textEmbedding"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {IAM_TOKEN}", "x-folder-id": f"{FOLDER_ID}"}

doc_texts = [
  """Александр Сергеевич Пушкин (26 мая [6 июня] 1799, Москва — 29 января [10 февраля] 1837, Санкт-Петербург) — русский поэт, драматург и прозаик, заложивший основы русского реалистического направления, литературный критик и теоретик литературы, историк, публицист, журналист.""",
  """Ромашка — род однолетних цветковых растений семейства астровые, или сложноцветные, по современной классификации объединяет около 70 видов невысоких пахучих трав, цветущих с первого года жизни."""
]

query_text = "когда день рождения Пушкина?"

def get_embedding(text: str, text_type: str = "doc") -> np.array:
    query_data = {
        "modelUri": doc_uri if text_type == "doc" else query_uri,
        "text": text,
    }

    return np.array(
        requests.post(embed_url, json=query_data, headers=headers).json()["embedding"]
    )


query_embedding = get_embedding(query_text, text_type="query")
docs_embedding = [get_embedding(doc_text) for doc_text in doc_texts]

# Вычисляем косинусное расстояние
dist = cdist(query_embedding[None, :], docs_embedding, metric="cosine")

# Вычисляем косинусное сходство
sim = 1 - dist

# most similar doc text
print(doc_texts[np.argmax(sim)])
